## ndarray的储存结构(tensor类似)

&emsp;&emsp;ndarray的储存结构在于数据存储与其解释方式的分离,或者说**copy**和**view**的分离,
即让尽可能多的操作发生在解释方式上(即**view**上),而尽量少地操作实际存储数据的内存区域.

解释方式主要包括:
* dtype:数据类型,指示了每个数据占用多少个字节,这几个字节怎么解释,如int32、float32
* ndim:有多少维
* shape:每维上的数量
* strides：维间距,即到达当前维下一个相邻数据需要前进的字节数,因考虑内存对齐,不一定为每个数据占用字节数的整数倍

上面4个信息构成了ndarray的indexing schema,即如何索引到指定位置的数据,以及这个数据该怎么解释

&emsp;&emsp;如下所示,a通过reshape操作返回的新的对象b,a和b的shape不同,但是两者共享同一个数据block;
c=b.T,c是b的转置,但两者仍共享同一个数据block,数据block并没有发生变化,发生变化的只是数据block的解释方式.

numpy中常见的共享数据物理地址(注:不是引用地址)的操作有(详情参考arr的修改与共享内存.ipynb):
1. reshape
2. ravel(注:flatten不共享内存)
3. 切片
4. 维度变换(.T,transpose,swapaxes)
5. 维度增减(expand_dims,squeeze)

torch中常见的共享内存的操作有(详情参考tensor的修改与内存共享):
1. reshape/view
2. flatten
3. detach
4. 切片
5. 维度变换(.T,transpose,permute)
6. 维度增减(unsqueeze,squeeze)

若要拷贝其副本:list/pandas/numpy参考深拷贝浅拷贝copy模块;torch参考复制clone-detach.ipynb

In [1]:
import numpy as np
from ctypes import string_at

# NumPy arrays consist of two major components, the raw array data (from now on, referred to as the data buffer),
# and the information about the raw array data.
a = np.arange(12) # np.arange(12)的物理地址中保存有数据的物理地址和对数据的解释方式;变量a保存的是np.arrange(12)的指针地址
b = a.reshape(3, 4) # 只是对数据的解释方式发生变化,数据物理地址相同;变量b保存的是a.reshape(3, 4)的指针地址
c = b.T
d = b[0:2, 2:]

# shape各不相同
print(a)
print(b)
print(c)
print(d)

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
[[2 3]
 [6 7]]


In [2]:
# 指向地址各不相同
print(id(a))
print(id(b))
print(id(c))
print(id(d))

2381683444992
2381664041200
2381688030112
2381688030192


In [3]:
# a,b,c,d共享内存(数据的物理地址相同)
print(np.shares_memory(a, b))
print(np.shares_memory(a, c))
print(np.shares_memory(a, d))

True
True
True


In [4]:
# a,b,c,d指向的数据在内存中存储结构(连续储存)
print(string_at(a.ctypes.data, a.nbytes).hex())
print(string_at(b.ctypes.data, b.nbytes).hex())
print(string_at(c.ctypes.data, c.nbytes).hex())
print(string_at(d.ctypes.data, d.nbytes).hex())

000000000100000002000000030000000400000005000000060000000700000008000000090000000a0000000b000000
000000000100000002000000030000000400000005000000060000000700000008000000090000000a0000000b000000
000000000100000002000000030000000400000005000000060000000700000008000000090000000a0000000b000000
02000000030000000400000005000000


&emsp;&emsp;ndarray是为矩阵运算服务的,ndarray中的所有数据都是同一种类型,比如int32、float64等,
每个数据占用的字节数相同、解释方式也相同,所以可以稠密地排列在一起,而list可以容纳不同类型的对象,像string、int、tuple等都可以放在一个list内,
所以list中存放的是对象的引用,再通过引用找到具体的对象,这些对象所在的物理地址并不是连续的,如下所示:

<img src="../../Other/img/ndarray_list数据储存.png">

所以相对ndarray,list访问到数据需要多跳转1次,list只能做到对对象引用的按秩访问,对具体的数据并不是按秩访问,所以效率上ndarray比list要快得多,
空间上,因为ndarray只把数据紧密存储,而list需要把每个对象的所有域值都存下来,所以ndarray比list要更省空间.